In [1]:
# Magic comands
%reload_ext autoreload
%autoreload 2

# Libraries
import re
import numpy as np
import pandas as pd
import os, sys, webcolors
from datetime import datetime
import plotly.graph_objects as go

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

origin_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(origin_path, os.pardir))
os.chdir(parent_dir)

from app.nqueens import NQueensGeneticAlgorithm, NQueensBacktracking, NQueensHillClimbing

## __1. Algorithm Functions__

### __1.1 Backtracking__

In [2]:
def backtracking_algorithm(n= 8):
    # initializing timer
    start_time = datetime.now()

    # algorithm
    backtracking = NQueensBacktracking()
    results = backtracking.solveNQueens(n)

    # stopping timer
    end_time = datetime.now()

    # calculating runtime in miliseconds
    execution_time = int((end_time - start_time).total_seconds() * 1000)

    return {'dimensions': n, 'duration': execution_time}

In [3]:
# testing
ba = backtracking_algorithm(n = 12)

ba

{'dimensions': 11, 'duration': 192}

### __1.2 Hill Climbing__

In [4]:
def hill_climbing(n=8):
    
    nQueensHC = NQueensHillClimbing(dimension=n)

    nQueensHC.run()

    dict_output = re.search(r"{.*}", str(nQueensHC.report()), re.DOTALL).group(0)

    result = eval(dict_output)

    return result

In [5]:
def hill_climbing(n=8):
    nQueensHC = NQueensHillClimbing(dimension=n)
    nQueensHC.run()
    return nQueensHC.report()


In [6]:
# testing
hc = hill_climbing(n = 12)

hc

Initial position: [8, 6, 10, 1, 4, 3, 5, 0, 2, 7, 9], fitness: 46
Found better solution: [8, 6, 10, 1, 4, 0, 5, 0, 2, 7, 9], Fitness: 49
Found better solution: [3, 6, 10, 1, 4, 0, 5, 0, 2, 7, 9], Fitness: 51
Found better solution: [3, 6, 10, 1, 4, 0, 5, 8, 2, 7, 9], Fitness: 53
Hill Climbing returned: [3, 6, 10, 1, 4, 0, 5, 8, 2, 7, 9], Fitness: 53


{'best_fitness': 53,
 'best_fitness_percentage': 96.36363636363636,
 'best_representation': [3, 6, 10, 1, 4, 0, 5, 8, 2, 7, 9]}

### __1.3 Simulated Annealing__

In [7]:
def sim_annealing():
    pass

In [8]:
def genetic_algorithm(
                    n = 12,
                    pop = 100,
                    gen = 50,
                    mut_perc = .05, # .01 .1
                    cross_perc = .75, # .6 to .9
                    mut='random_mutation',
                    cross='single_cross',
                    select='tournament_selection',
                    ):
    
    nQueensGA = NQueensGeneticAlgorithm(population_size=pop, dimension=n)

    nQueensGA.run(
        generations=gen,
        mutation_probability=mut_perc,
        crossover_probability=cross_perc,
        mutation_operator=mut,
        crossover_operator=cross,
        selection_operator=select
    )

    return nQueensGA.report()

In [23]:
# testing function
ga = genetic_algorithm(
                                            n = 12,
                                            pop = 100,
                                            gen = 100,
                                            mut_perc = .05,
                                            cross_perc = .75,
                                            mut = "inversion_mutation",
                                            cross = "arithmetic_cross",
                                            select = "rank"
                                            )
ga

{'generations': 100,
 'duration': 152,
 'best_fitness': 64,
 'best_fitness_percentage': 96.96969696969697,
 'best_representation': [4, 5, 5, 6, 7, 4, 4, 7, 7, 6, 7, 3],
 'worst_fitness': 51,
 'worst_representation': [6, 6, 5, 6, 5, 5, 6, 6, 6, 5, 5, 5],
 'mean_fitness': 51.63,
 'selection_operator': 'rank_selection',
 'mutate_operator': 'inversion_mutation',
 'crossover_operator': 'arithmetic_xo'}

## __1. Operators & Paramaters Optimization__

In [10]:
# dictionary with parameter names and default values 
defaults = genetic_algorithm.__defaults__
parameters = genetic_algorithm.__code__.co_varnames
params = {param: val for param, val in zip(parameters, defaults)}

# color list for plots
color_palette = ['gray', 'steelblue', 'purple', 'darkgray', 'lightgray', 'slategray', 'cornflowerblue', 'lavender', 'powderblue']

In [11]:
def best_operators_search(parameter: str, values: list, previous_best_params: list = None):
    
    if previous_best_params:
        params = previous_best_params
    else:
        # dictionary with parameter names and default values 
        defaults = genetic_algorithm.__defaults__
        parameters = genetic_algorithm.__code__.co_varnames
        params = {param: val for param, val in zip(parameters, defaults)}
    
    # initializing variables
    best_fit = 0.0
    time_fit = 9e99
    fit_dict = {}
    time_dict = {}
    param_list = []

    # Perform grid search
    for i in values:
        # 100 iterations to generate statistically relevant results
        fit_list = []
        time_list = []
        for _ in range(100):
            # changing parameter according to input
            params[parameter] = i
            # Define the model with current hyperparameters
            ga = genetic_algorithm(**params)
            # gets restults
            fit = ga['best_fitness_percentage']
            time_it = ga['duration']
            # Check if current solution has better fitness than previous best 
            if fit > best_fit:
                best_fit = fit
                time_fit = time_it
                best_param = {parameter: i}
            # Check if the solution with fitness as good is faster than previous best
            if fit == best_fit and time_it < time_fit:
                time_fit = time_it
                best_param = {parameter: i}
            # Append results to get 100 iterations values
            fit_list.append(fit)
            time_list.append(time_it)
        # saving results per parameter
        param_list.append(best_param)
        fit_dict['Fit_' + str(i)] = fit_list
        time_dict['Time_' + str(i)] = time_list

    unique_best_params =list({tuple(d.items()): d for d in param_list}.values())

    # dataframe with results
    fit_dict.update(time_dict)
    df = pd.DataFrame(fit_dict)
        
    return unique_best_params, df

In [12]:
def plot_values(data: pd.DataFrame, colors: list, layout: int, template:str, grid:  bool):
    # Filter columns starting with "Fit_"
    fit_columns = [col for col in data.columns if col.startswith('Fit_')]

    # Filter columns starting with "Time_"
    time_columns = [col for col in data.columns if col.startswith('Time_')]

    template_map = {'white':'plotly_white', 'dark': 'plotly_dark'}

    tick_font_map = {'white': webcolors.rgb_to_hex(webcolors.name_to_rgb('black')), 'dark': webcolors.rgb_to_hex(webcolors.name_to_rgb('white'))}

    # Layout styles
    layout_1 = {
        'xaxis_title': 'Iterations',
        'yaxis_title': 'Time (ms)',
        'template': template_map[template],
        'font': {'size': 14, 'family': 'Verdana'},
        'title': {'text': 'Time Comparison', 'font': {'size': 19, 'family': 'Verdana'}},
        'xaxis': {'showgrid': grid, 'ticks': 'outside', 'tickfont': {'color': tick_font_map[template]}},
        'yaxis': {'showgrid': grid, 'tickfont': {'color': tick_font_map[template]}}
        }
    
    layout_2 = {
        'template': template_map[template],
        'font': {'size': 14, 'family': 'Verdana'},
        'xaxis': {
            'showline': True,
            'showgrid': grid,
            'showticklabels': True,
            'linewidth': 1.5,
            'ticks': 'outside',
            'tickfont': {
                'family': 'Verdana',
                'size': 11,
                'color': tick_font_map[template]}},
        'yaxis': {
            'showgrid': grid,
            'zeroline': False,
            'showline': False,
            'showticklabels': True,
            'ticks': 'outside',
            'tickfont': {
                'family': 'Verdana',
                'size': 11,
                'color': tick_font_map[template]}}}
    
    layout_map = {1:layout_1, 2:layout_2}

    # Convert color names to RGB values
    rgb_colors = [webcolors.name_to_rgb(color) for color in colors]

    mode_size = 8
    line_size = 2

    # Create a subplot with two line plots
    fig = go.Figure()

    # Add line plots for "Fit_" columns
    i = 0
    for col in fit_columns:
        fig.add_trace(go.Scatter(x=data.index, y=data[col], name=col.replace('Fit_', ''),
                                line=dict(color=colors[i], width=line_size),
                                ))

        fig.add_trace(go.Scatter(
            x=[data.index[0], data.index[-1]],
            y=[data[col][0], data[col].iloc[-1]],
            mode='markers',
            marker=dict(color=colors[i], size=mode_size),
            name=''
        ))
        i += 1

    layout_map[layout].update({
                            'title': 'Fitness Comparison',
                            'xaxis_title': 'Iterations',
                            'yaxis_title': 'Fitness'})

    fig.update_layout(layout_map[layout])

    fig2 = go.Figure()

    # Add line plots for "Time_" columns
    i = 0
    for col in time_columns:
        fig2.add_trace(go.Scatter(x=data.index, y=data[col], name=col.replace('Time_', ''),
                                line=dict(color=colors[i], width=line_size),
                                ))

        fig2.add_trace(go.Scatter(
            x=[data.index[0], data.index[-1]],
            y=[data[col][0], data[col].iloc[-1]],
            mode='markers',
            marker=dict(color=colors[i], size=mode_size),
            name=''
        ))
        i += 1

    layout_map[layout].update({
                            'title': 'Time Comparison',
                            'xaxis_title': 'Iterations',
                            'yaxis_title': 'Time (ms)'})

    fig2.update_layout(layout_map[layout])

    # Show the plots
    fig.show()
    fig2.show()

### __2.1 Mutation Operator__

In [13]:
# seed for reproducibility
np.random.seed(0)

# Define the range of values
mutation = ['swap_mutation', 'random_mutation', 'inversion_mutation']

# Print current parameters to use in best_operators_search
print('\033[1mParameters for GA:\033[0m', end='\n\n')
print(params, end='\n\n')

param_list, df = best_operators_search('mut', mutation, params)

# Displaying average results of the 100 iterations
print('\033[1mAverage results:', end='\n')
df['const'] = 'const'
df_avg = df.groupby('const').mean().reset_index(drop=True).round(3)
display(df_avg)

# Getting best parameter by fitness & time
best_param_fit = df_avg[[col for col in df_avg.columns if col.startswith('Fit_')]].idxmax(axis=1)[0].replace('Fit_', '')
best_param_time = df_avg[[col for col in df_avg.columns if col.startswith('Time_')]].idxmin(axis=1)[0].replace('Time_', '')
print(f'\nBest Fitness: \033[1m{best_param_fit}\033[0m\n\nBest Time: \033[1m{best_param_time}\033[0m', end='\n\n')

# Setting the best parameter as new default value
replace_param = {'mut': best_param_time}

# Plotting results
plot_values(data = df, colors = color_palette[:len(mutation)], layout=2, template='white', grid=True)

Parameters for GA:

{'n': 25, 'pop': 100, 'gen': 50, 'mut_perc': 0.05, 'cross_perc': 0.75, 'mut': 'random_mutation', 'cross': 'single_cross', 'select': 'tournament_selection'}

starting loop
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79

,Fit_swap_mutation,Fit_random_mutation,Fit_inversion_mutation,Time_swap_mutation,Time_random_mutation,Time_inversion_mutation
0,97.34,97.367,97.367,10.11,11.27,10.04



Best Fitness: random_mutation

Best Time: inversion_mutation



### __2.2 Crossover Operator__

In [14]:
# seed for reproducibility
np.random.seed(0)

# Define the range of values
xo = ['single_cross', 'cycle_cross', 'pmx', 'arithmetic_cross']

# Setting previous best parameter as new default value
params.update(replace_param)

# Print current parameters to use in best_operators_search
print('\033[1mParameters for GA:\033[0m', end='\n\n')
print(params, end='\n\n')

param_list, df = best_operators_search('cross', xo, params)

# Displaying average results of the 100 iterations
print('\033[1mAverage results:', end='\n')
df['const'] = 'const'
df_avg = df.groupby('const').mean().reset_index(drop=True).round(3)
display(df_avg)

# Getting best parameter by fitness & time
best_param_fit = df_avg[[col for col in df_avg.columns if col.startswith('Fit_')]].idxmax(axis=1)[0].replace('Fit_', '')
best_param_time = df_avg[[col for col in df_avg.columns if col.startswith('Time_')]].idxmin(axis=1)[0].replace('Time_', '')
replace_param = {'cross': best_param_fit}
print(f'\nBest Fitness: \033[1m{best_param_fit}\033[0m\n\nBest Time: \033[1m{best_param_time}\033[0m', end='\n\n')

# Plotting results
plot_values(data = df, colors = color_palette[:len(xo)], layout=2, template='white', grid=True)

Parameters for GA:

{'n': 25, 'pop': 100, 'gen': 50, 'mut_perc': 0.05, 'cross_perc': 0.75, 'mut': 'inversion_mutation', 'cross': 'single_cross', 'select': 'tournament_selection'}

starting loop
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78

,Fit_single_cross,Fit_cycle_cross,Fit_pmx,Fit_arithmetic_cross,Time_single_cross,Time_cycle_cross,Time_pmx,Time_arithmetic_cross
0,97.297,97.32,97.35,97.367,10.06,16.91,13.72,20.19



Best Fitness: arithmetic_cross

Best Time: single_cross



### __2.3 Selection Operator__

In [15]:
# seed for reproducibility
np.random.seed(0)

# Define the range of values
select = ['tournament_selection', 'fps', 'rank']

# Setting previous best parameter as new default value
params.update(replace_param)

# Print current parameters to use in best_operators_search
print('\033[1mParameters for GA:\033[0m', end='\n\n')
print(params, end='\n\n')

param_list, df = best_operators_search('select', select, params)

# Displaying average results of the 100 iterations
print('\033[1mAverage results:', end='\n')
df['const'] = 'const'
df_avg = df.groupby('const').mean().reset_index(drop=True).round(3)
display(df_avg)

# Getting best parameter by fitness & time
best_param_fit = df_avg[[col for col in df_avg.columns if col.startswith('Fit_')]].idxmax(axis=1)[0].replace('Fit_', '')
best_param_time = df_avg[[col for col in df_avg.columns if col.startswith('Time_')]].idxmin(axis=1)[0].replace('Time_', '')
replace_param = {'select': best_param_fit}
print(f'\nBest Fitness: \033[1m{best_param_fit}\033[0m\n\nBest Time: \033[1m{best_param_time}\033[0m', end='\n\n')

# Plotting results
plot_values(data = df, colors = color_palette[:len(select)], layout=2, template='white', grid=True)

Parameters for GA:

{'n': 25, 'pop': 100, 'gen': 50, 'mut_perc': 0.05, 'cross_perc': 0.75, 'mut': 'inversion_mutation', 'cross': 'arithmetic_cross', 'select': 'tournament_selection'}

starting loop
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
7

,Fit_tournament_selection,Fit_fps,Fit_rank,Time_tournament_selection,Time_fps,Time_rank
0,97.283,97.297,97.367,19.98,63.97,74.09



Best Fitness: rank

Best Time: tournament_selection



### __2.4 Population__

In [16]:
# seed for reproducibility
np.random.seed(0)

# Define the range of values
population = [100, 250, 500]

# Setting previous best parameter as new default value
params.update(replace_param)

# Print current parameters to use in best_operators_search
print('\033[1mParameters for GA:\033[0m', end='\n\n')
print(params, end='\n\n')

param_list, df = best_operators_search('pop', population, params)

# Displaying average results of the 100 iterations
print('\033[1mAverage results:', end='\n')
df['const'] = 'const'
df_avg = df.groupby('const').mean().reset_index(drop=True).round(3)
display(df_avg)

# Getting best parameter by fitness & time
best_param_fit = df_avg[[col for col in df_avg.columns if col.startswith('Fit_')]].idxmax(axis=1)[0].replace('Fit_', '')
best_param_time = df_avg[[col for col in df_avg.columns if col.startswith('Time_')]].idxmin(axis=1)[0].replace('Time_', '')
replace_param = {'pop': best_param_fit}
print(f'\nBest Fitness: \033[1m{best_param_fit}\033[0m\n\nBest Time: \033[1m{best_param_time}\033[0m', end='\n\n')

# Plotting results
plot_values(data = df, colors = color_palette[:len(population)], layout=2, template='white', grid=True)

Parameters for GA:

{'n': 25, 'pop': 100, 'gen': 50, 'mut_perc': 0.05, 'cross_perc': 0.75, 'mut': 'inversion_mutation', 'cross': 'arithmetic_cross', 'select': 'rank'}

starting loop
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


KeyboardInterrupt: 

### __2.5 Generations__

In [16]:
# seed for reproducibility
np.random.seed(0)

generations = [50, 100, 250, 500]

# Setting previous best parameter as new default value
params.update(replace_param)

# Print current parameters to use in best_operators_search
print('\033[1mParameters for GA:\033[0m', end='\n\n')
print(params, end='\n\n')

param_list, df = best_operators_search('gen', generations, params)

# Displaying average results of the 100 iterations
print('\033[1mAverage results:', end='\n')
df['const'] = 'const'
df_avg = df.groupby('const').mean().reset_index(drop=True).round(3)
display(df_avg)

# Getting best parameter by fitness & time
best_param_fit = df_avg[[col for col in df_avg.columns if col.startswith('Fit_')]].idxmax(axis=1)[0].replace('Fit_', '')
best_param_time = df_avg[[col for col in df_avg.columns if col.startswith('Time_')]].idxmin(axis=1)[0].replace('Time_', '')
replace_param = {'gen': best_param_fit}
print(f'\nBest Fitness: \033[1m{best_param_fit}\033[0m\n\nBest Time: \033[1m{best_param_time}\033[0m', end='\n\n')

# Plotting results
plot_values(data = df, colors = color_palette[:len(generations)], layout=2, template='white', grid=True)

Parameters for GA:

{'n': 25, 'pop': 100, 'gen': 50, 'mut_perc': 0.05, 'cross_perc': 0.75, 'mut': 'inversion_mutation', 'cross': 'pmx', 'select': 'fps'}

Average results:


,Fit_50,Fit_100,Fit_250,Fit_500,Time_50,Time_100,Time_250,Time_500
0,97.32,97.31,97.303,97.36,58.1,116.48,297.39,601.41



Best Fitness: 500

Best Time: 50



## __3. Algortihms Comparison__